In [ ]:
'''
Date: 2019/12/30
OS: Window 10

Tensorflow-gpu: v1.13.1

LAST UPDATE: 2020/01/10
'''

In [1]:
# Import packages
import os
import cv2
import numpy as np
import tensorflow as tf
import sys

# This is needed since some modules are stored in the object_detection folder.
sys.path.append("D:/課程/北大課程/人工神經網路與深度學習/Project04/models/research")

# Import utilites
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as vis_util

class color:
    PURPLE = '\033[95m'
    CYAN = '\033[96m'
    DARKCYAN = '\033[36m'
    BLUE = '\033[94m'
    GREEN = '\033[92m'
    YELLOW = '\033[93m'
    RED = '\033[91m'
    BOLD = '\033[1m'
    UNDERLINE = '\033[4m'
    END = '\033[0m'

c:\users\leonp\anaconda3\envs\tfv1131\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
c:\users\leonp\anaconda3\envs\tfv1131\lib\site-packages\tensorflow\python\framework\dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
c:\users\leonp\anaconda3\envs\tfv1131\lib\site-packages\tensorflow\python\framework\dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
c:\users\leonp\anaconda3\envs\tfv1131\lib\site-packages\tensorflow\

In [2]:
# Name of the directory containing the object detection module we're using
# MODEL_NAME = 'faster_rcnn_inception_v2_coco_2018_01_28'

# Grab path to current working directory
# CWD_PATH = os.getcwd()
CWD_PATH = "D:\\課程\\北大課程\\人工神經網路與深度學習\\Project04\\models\\research\\object_detection"

# Path to frozen detection graph .pb file, which contains the model that is used for object detection.
PATH_TO_CKPT = os.path.join(CWD_PATH, "inference_graph", 'frozen_inference_graph.pb')

# Path to label map file
PATH_TO_LABELS = os.path.join(CWD_PATH, 'training', 'labelmap.pbtxt')

# Path to image
PATH_TO_IMAGE = "D:\\test_cdc\\test_images"

# Path to output dir
PATH_TO_OUTPUT = "D:\\課程\\北大課程\\人工神經網路與深度學習\\Project04"

# Number of classes the object detector can identify
NUM_CLASSES = 13

print("Cwd: {}\n".format(CWD_PATH))
print("Checkpoint: \n   {}\n".format(PATH_TO_CKPT))
print("Label map: {}\n".format(PATH_TO_LABELS))
print("Image dir: {}".format(PATH_TO_IMAGE))

Cwd: D:\課程\北大課程\人工神經網路與深度學習\Project04\models\research\object_detection

Checkpoint: 
   D:\課程\北大課程\人工神經網路與深度學習\Project04\models\research\object_detection\inference_graph\frozen_inference_graph.pb

Label map: D:\課程\北大課程\人工神經網路與深度學習\Project04\models\research\object_detection\training\labelmap.pbtxt

Image dir: D:\test_cdc\test_images


In [3]:
def bulid_session(PATH_TO_LABELS, PATH_TO_CKPT, NUM_CLASSES):
    # Load the label map.
    # Here we use internal utility functions, but anything that returns a
    # dictionary mapping integers to appropriate string labels would be fine
    label_map = label_map_util.load_labelmap(PATH_TO_LABELS)
    categories = label_map_util.convert_label_map_to_categories(label_map, max_num_classes=NUM_CLASSES, use_display_name=True)
    category_index = label_map_util.create_category_index(categories)

    # Load the Tensorflow model into memory.
    detection_graph = tf.Graph()                               # assign tf.Graph() to detection_graph
    with detection_graph.as_default():                         # set detection_graph as default
        od_graph_def = tf.GraphDef()                           # serialize the tf.Graph()
        with tf.gfile.GFile(PATH_TO_CKPT, 'rb') as fid:        # get checkpoint file in binary data as fid
            serialized_graph = fid.read()                      # read data
            od_graph_def.ParseFromString(serialized_graph)     # uncode the serialized graph
            tf.import_graph_def(od_graph_def, name='')         # put the uncode data back to od_graph_def

        sess = tf.Session(graph=detection_graph)               # open tf.Session(graph=detection_graph) and assign it to sess

    ## Define input and output tensors (i.e. data) for the object detection classifier
    # Input tensor is the image
    image_tensor = detection_graph.get_tensor_by_name('image_tensor:0')

    # Output tensors are the detection boxes, scores, and classes
    # Each box represents a part of the image where a particular object was detected
    detection_boxes = detection_graph.get_tensor_by_name('detection_boxes:0')

    # Each score represents level of confidence for each of the objects.
    # The score is shown on the result image, together with the class label.
    detection_scores = detection_graph.get_tensor_by_name('detection_scores:0')
    detection_classes = detection_graph.get_tensor_by_name('detection_classes:0')

    # Number of objects detected
    num_detections = detection_graph.get_tensor_by_name('num_detections:0')

    # pack all outputs in detections
    detections = [detection_boxes, 
                  detection_scores, 
                  detection_classes, 
                  num_detections]

    return sess, detections, image_tensor, category_index

In [4]:
def predict_single_image(imgPath, sess, detections, image_tensor):
    # Load image using OpenCV and expand image dimensions to have shape: [1, None, None, 3]
    # i.e. a single-column array, where each item in the column has the pixel RGB value
    image = cv2.imread(imgPath)
    height, width, channels = image.shape
    image_expanded = np.expand_dims(image, axis=0)

    # Perform the actual detection by running the model with the image as input
    # The order of boxes is [ymin, xmin, ymax, xmax]
    boxes, scores, classes, num = sess.run(detections, 
                                           feed_dict={image_tensor: image_expanded})

    return width, height, np.squeeze(boxes), np.squeeze(classes), np.squeeze(scores)

In [5]:
def display_single_image(imgPath, boxes, classes, scores):
    image = cv2.imread(imgPath)
    vis_util.visualize_boxes_and_labels_on_image_array(
        image,
        boxes,
        classes.astype(np.int32),
        scores,
        category_index,
        use_normalized_coordinates=True,
        line_thickness=8,
        min_score_thresh=0.60)

    ## Keep in mind that cv2.imshow() always comes with cv2.waitKey()
    # All the results have been drawn on image. Now display the image.
    cv2.imshow('Object detector', image)
    print(color.RED + color.BOLD + "Press any key to close the image" + color.END + color.END)

    # Press any key to close the image
    cv2.waitKey(0)

    # Clean up
    cv2.destroyAllWindows()

In [28]:
def display_four_images(imgPath, labelPath, checkpointPath, num_labels):
    sess, detections, image_tensor, category_index = bulid_session(labelPath, checkpointPath, num_labels)
    imgs = os.listdir(imgPath)

    IMAGE = []
    # pick 4 random images from all test images
    for img_idx in np.random.choice(range(len(imgs)), 4):
        path = os.path.join(PATH_TO_IMAGE, imgs[img_idx])
        image = cv2.imread(path)
        width, height, boxes, labels, prob = predict_single_image(path, sess, detections, image_tensor)

        # Draw the results of the detection aka "visulaize the results"
        vis_util.visualize_boxes_and_labels_on_image_array(
            image,
            boxes,
            labels.astype(np.int32),
            prob,
            category_index,
            use_normalized_coordinates=True,
            line_thickness=8,
            min_score_thresh=0.60)

        # resize the image
        image_adj = cv2.resize(image, (400, 300), interpolation=cv2.INTER_CUBIC)
        IMAGE.append(image_adj)

    # concatenate the images and show them all at the same time
    img_row1 = np.concatenate((IMAGE[0], IMAGE[1]), axis=1)
    img_row2 = np.concatenate((IMAGE[2], IMAGE[3]), axis=1)
    img = np.concatenate((img_row1, img_row2), axis=0)
    cv2.imshow('Object detector', img)
    print(color.RED + color.BOLD + "Press any key to close the window" + color.END + color.END)

    sess.close()

    ## Keep in mind that cv2.imshow() always comes with cv2.waitKey()
    # Press any key to close the image
    cv2.waitKey(0)
    # Clean up
    cv2.destroyAllWindows()

In [ ]:
print("The boxes order: " + color.RED + color.BOLD + "[ymin, xmin, ymax, xmax]" + color.END + color.END)

def lines_for_single_image(img_width, img_height, boxes, labels, scores, imgName, default_rate=0.6):
    # pick the object according to the probability
    num_pick = sum(scores > default_rate)
    
    # follow the submission example order: x y w h
    obj_left_border = boxes[:, 1] * img_width                        # top-left x value of the object
    obj_top_border = boxes[:, 2] * img_height                        # top-left y value of the object
    obj_width = (boxes[:, 3] - boxes[:, 1]) * img_width              # width of the object
    obj_height = (boxes[:, 2] - boxes[:, 0]) * img_height            # height of the object

    lines = ""
    for i in range(num_pick):
        line = imgName + ",{:0.0f},{},{},{},{},{:.9f}".format(labels[i], 
                                                              obj_left_border[i], 
                                                              obj_top_border[i], 
                                                              obj_width[i], 
                                                              obj_height[i],
                                                              scores[i])
        lines += line + ("\n" if i < (num_pick-1) else "")
    return lines

In [ ]:
def output_as_csv(imgPath, outputPath, labelPath, checkpointPath, num_labels):
    LINES = ""
    images_list = os.listdir(imgPath)
    images_total = len(images_list)
    
    # create session
    sess, detections, image_tensor, category_index = bulid_session(labelPath, checkpointPath, num_labels)
    
    count = 0
    for img in images_list:
        if img.endswith(".jpg"):
            # get one image path
            path = os.path.join(imgPath, img)
            # predict the result
            width, height, boxes, labels, prob = predict_single_image(path, sess, detections, image_tensor)
            # adjust prediction for output as csv file
            lines = lines_for_single_image(width, height, boxes, labels, prob, img)

            # print out the current iteration situation
            count += 1
            print("{}/{} images - The file being processing is: {}\t".format(count, images_total, img), end="\r")
            
            LINES += lines
        LINES += ("\n" if lines != "" else "")
    
    # close session so that it will release the memory
    sess.close()
    if sess._closed:
        print("\nSession Closed")
    
    print("Writing csv file ... ", end="")
    with open(os.path.join(outputPath, "output.csv"), "w") as csv_file:
        print("image_filename,label_id,x,y,w,h,confidence", file = csv_file)
        print(LINES, file = csv_file)
    print(color.BOLD + "Finished" + color.END)

In [29]:
# better plot some prediction before output as csv
# since you might find your model was not good enough for prediction
display_four_images(PATH_TO_IMAGE, PATH_TO_LABELS, PATH_TO_CKPT, NUM_CLASSES)

Press any key to close the image


In [ ]:
output_as_csv(PATH_TO_IMAGE, PATH_TO_OUTPUT, PATH_TO_LABELS, PATH_TO_CKPT, NUM_CLASSES)